In [1]:
import numpy as np, pickle
from IPython.display import Audio
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import featurizer, data_formatter, models
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics.pairwise import cosine_similarity

Using cuDNN version 5105 on context None
Mapped name None to device cuda: Tesla K40c (0000:02:00.0)


Loading model parameters...
Compiling encoders...
Loading tables...
Packing up...


Using TensorFlow backend.
/home/jrgillick/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/jrgillick/.local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
with open('/data/jrgillick/speeches.pkl') as f:
    speeches = pickle.load(f)

In [3]:
import time
t0 = time.time()
with open('/data/jrgillick/individual_feature_lists.pkl') as f:
    individual_feature_lists = pickle.load(f)
print "load in " + str(time.time() - t0)

load in 903.656893969


In [1159]:
#with open('/data/jrgillick/speeches.pkl','wb') as f:
#    pickle.dump(speeches,f)

In [4]:
speaker_names = ['donald_trump',
 'hilary_clinton',
 'bernie_sanders',
 'ted_cruz',
 'marco_rubio',
 'john_kasich',
 'barack_obama',
 'bill_clinton',
 'joe_biden',
 'mike_pence',
 'carly_fiorina',
 'jeb_bush',
 'rand_paul',
 'gary_johnson',
 'chris_christie',
 'rick_santorum']

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [237]:
#for s in speeches:
#    phrase_list = s.alignment.get_phrase_text()
#    with open('/data/corpora/cspan/phrase_lists/' + s.file_path.split('/')[1] + '.txt','wb') as f:
#        for p in phrase_list:
#            f.write(p + '\n')

In [301]:
speeches = [speech.Speech(s.file_path.split('/')[1]) for s in tqdm(speeches)]

100%|██████████| 310/310 [30:49<00:00,  5.54s/it]


In [302]:
for s in speeches:
    s.phrase_audio_features = s.get_phrase_audio_features()
    normalize_phrase_audio_features(s)

In [251]:
for s in speeches:
    try:
        s.normalize_phrase_audio_features()
    except:
        print s.file_path

mike_pence/mike_pence_4
barack_obama/barack_obama_6
gary_johnson/gary_johnson_2
jeb_bush/jeb_bush_4
donald_trump/donald_trump_114
donald_trump/donald_trump_75
donald_trump/donald_trump_20
bernie_sanders/bernie_sanders_14
bernie_sanders/bernie_sanders_44
bernie_sanders/bernie_sanders_16
hilary_clinton/hilary_clinton_13


In [304]:
train_speeches, test_speeches = train_test_split(speeches, test_size=0.2, random_state=43)

In [305]:
len(train_speeches)

248

In [306]:
len(test_speeches)

62

In [ ]:
def normalize_data(X_train, X_test):
    new_X_train = np.copy(X_train)
    new_X_test = np.copy(X_test)
    for i in xrange(new_X_train.shape[1]):
        if set(new_X_train[:,i]) != set([0,1]) and set(new_X_train[:,i]) != set([0]) and set(new_X_train[:,i]) != set([1]):
            #print 'norming'
            #print set(new_X_train[:,i])
            mean = np.mean(new_X_train[:,i])
            new_X_train[:,i] -= mean
            new_X_test[:,i] -=mean
            std = np.std(X_train[:,i])
            if std > 1e-2:
                new_X_train[:,i] /= std
                new_X_test[:,i] /= std
    return new_X_train, new_X_test

In [402]:
reload(data_formatter)
reload(featurizer)

<module 'featurizer' from 'featurizer.py'>

In [ ]:
train_bigram_feats = data_formatter.get_bigram_data(train_speeches)

100%|██████████| 248/248 [1:12:06<00:00, 14.34s/it]


61

In [464]:
test_bigram_feats = data_formatter.get_bigram_data(test_speeches)


  0%|          | 0/62 [00:00<?, ?it/s]

100%|██████████| 62/62 [17:53<00:00, 17.33s/it]


In [472]:
speech_names = [s.file_path for s in speeches]

In [972]:
individual_feature_lists = []
for i in range(len(train_speeches)):
    h = {'id': train_speeches[i].file_path,
         'bigrams': train_bigram_feats[i],
         'rst': train_rst_feats[i],
         'audio': train_audio_feats[i],
         'euphony': train_euphony_feats[i],
         'liu': train_liu_feats[i],
         'cosine': np.array(train_cosine_feats[i]).reshape(-1,1),
         'phone': train_phone_feats[i],
         'substring': np.array(train_substring_feats[i]).reshape(-1,1),
         'word_overlap': np.array(train_word_overlap_feats[i]).reshape(-1,1),
         'labels': train_labels[i]}
    individual_feature_lists.append(h)
    
for i in range(len(test_speeches)):
    h = {'id': test_speeches[i].file_path,
         'bigrams': test_bigram_feats[i],
         'rst': test_rst_feats[i],
         'audio': test_audio_feats[i],
         'euphony': test_euphony_feats[i],
         'liu': test_liu_feats[i],
         'cosine': np.array(test_cosine_feats[i]).reshape(-1,1),
         'phone': test_phone_feats[i],
         'substring': np.array(test_substring_feats[i]).reshape(-1,1),
         'word_overlap': np.array(test_word_overlap_feats[i]).reshape(-1,1),
         'labels': test_labels[i]}
    individual_feature_lists.append(h)

In [974]:
"""
c = 0
for i in range(len(test_rst_feats)):
    for j in range(len(test_rst_feats[i])):
        for k in range(len(test_rst_feats[i][j])):
            if test_rst_feats[i][j][k] != test_rst_feats2[i][j][k]:
                test_rst_feats[i][j][k] = test_rst_feats2[i][j][k]
    

c = 0
for i in range(len(train_rst_feats)):
    for j in range(len(train_rst_feats[i])):
        for k in range(len(train_rst_feats[i][j])):
            if train_rst_feats[i][j][k] != train_rst_feats2[i][j][k]:
                train_rst_feats[i][j][k] = train_rst_feats2[i][j][k]
"""

'\nc = 0\nfor i in range(len(test_rst_feats)):\n    for j in range(len(test_rst_feats[i])):\n        for k in range(len(test_rst_feats[i][j])):\n            if test_rst_feats[i][j][k] != test_rst_feats2[i][j][k]:\n                test_rst_feats[i][j][k] = test_rst_feats2[i][j][k]\n    \n\nc = 0\nfor i in range(len(train_rst_feats)):\n    for j in range(len(train_rst_feats[i])):\n        for k in range(len(train_rst_feats[i][j])):\n            if train_rst_feats[i][j][k] != train_rst_feats2[i][j][k]:\n                train_rst_feats[i][j][k] = train_rst_feats2[i][j][k]\n'

In [975]:
#test_rst_feats_old = np.copy(test_rst_feats)
#train_rst_feats_old = np.copy(train_rst_feats)

In [1030]:
with open('/data/jrgillick/individual_feature_lists.pkl','wb') as f:
    pickle.dump(individual_feature_lists,f)

In [1031]:
5

5

In [ ]:
def get_train_and_test_sets(speaker_name):
    train_set = []
    test_set = []
    for i in range(len(individual_feature_lists)):
        if speaker_name in individual_feature_lists[i]['id']:
            test_set.append(individual_feature_lists[i])
        else:
            train_set.append(individual_feature_lists[i])
    return train_set, test_set

In [36]:
def get_features(data_set, feature_names):
    all_feats = []
    for i in range(len(data_set)):
        feats = [data_set[i][name] for name in feature_names]
        all_feats.append(np.hstack(feats))
    return all_feats

def get_labels(data_set):
    all_labels = []
    for i in range(len(data_set)):
        all_labels.append(data_set[i]['labels'])
    return all_labels

In [38]:
len(speeches)

NameError: name 'speeches' is not defined

In [46]:
len(individual_feature_lists[0:248])

248

In [133]:
feature_set = ['bigrams','audio','liu','euphony','phone','cosine','substring','word_overlap','rst']
phrase_count = 1

train_set = individual_feature_lists[0:248]
test_set = individual_feature_lists[248:]

labels_train = get_labels(train_set)
data_train = get_features(train_set,feature_set)

labels_test = get_labels(test_set)
data_test = get_features(test_set,feature_set)

current_train_formatted_feats, current_train_formatted_labs = models.format_balanced_multiple_phrase_input(data_train,labels_train,phrase_count=phrase_count)
current_test_formatted_feats, current_test_formatted_labs = models.format_balanced_multiple_phrase_input(data_test,labels_test,phrase_count=phrase_count)

X_train = np.array(current_train_formatted_feats).astype('float64')
y_train = np.array(current_train_formatted_labs)
        
print X_train.shape

X_test = np.array(current_test_formatted_feats).astype('float64')
y_test = np.array(current_test_formatted_labs)

#X_train, X_test = normalize_data(X_train,X_test)
#model = models.train_cv_logistic_regression_faster(X_train,y_train)

(28832, 26859)


In [134]:
X_train, jeb_train = normalize_data(X_train, np.array(jeb_feats))

In [104]:
coefs = model.best_estimator_.coef_[0]
#coefs

In [106]:
import jeb_speech, jeb_featurizer

In [107]:
jeb = jeb_speech.Speech('jeb')
f = jeb_featurizer.Featurizer(jeb)

In [128]:
#feature_set = ['bigrams','audio','liu','euphony','phone','cosine','substring','word_overlap','rst']
jeb_feats =np.hstack([np.array(f.get_bigram_features()),
           np.array(f.get_audio_features()),
           np.array(f.get_liu_features()),
           np.array(f.get_euphony_features()),
           np.array(f.get_phone_features()),
           np.array(f.get_vector_cosine_distances()).reshape(-1,1),
           np.array(f.get_common_substring_features()).reshape(-1,1),
           np.array(f.get_n_common_words()).reshape(-1,1),
           np.array(f.get_rst_features()),
          ])



In [153]:
jeb_order_0 = np.argsort(jeb_train[0] * coefs)
jeb_order_1 = np.argsort(jeb_train[1] * coefs)
jeb_order_2 = np.argsort(jeb_train[2] * coefs)

In [155]:
jeb_train.shape

(3, 26859)

In [156]:
coefs

array([ 0.09509445,  0.04586393,  0.0455949 , ..., -0.05811358,
        0.00184465, -0.1136884 ])

In [162]:
h = {'coefs':coefs,'feature_names':all_feature_names,'jeb_features':jeb_train}
jeb_train

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  1.,  1.,  1.]])

In [165]:
with open('/data/corpora/cspan/jeb_coefs.pkl','wb') as f:
    pickle.dump(h,f)

In [166]:
with open('/data/corpora/cspan/jeb_coefs.pkl','rb') as f:
    h = pickle.load(f)

In [168]:
h['jeb_features'].shape

(3, 26859)

In [272]:
instances = s.applause_list.get_instances()

In [273]:
len(instances)

73

In [275]:
i = 0

In [274]:
s.load_librosa()

In [299]:
i +=1
Audio(s.y[int(instances[i][0]*s.sr):int(instances[i][1]*s.sr)], rate=s.sr)

In [298]:
i

19

In [ ]:
77/80

In [29]:
s = speeches[29]

In [30]:
s.file_path

'john_kasich/john_kasich_21'

In [31]:
times = s.alignment.get_phrase_times()
labs = [s.alignment.applause_follows(t[1]) for t in times]
zip(s.alignment.get_phrase_text(),labs)

[(u'IT JUST', False),
 (u'UNDERSCORES THE AMAZING LIFE AND JOURNEY THAT I HAD IN MY LIFETIME AS A',
  False),
 (u'QUARTER FRESHMAN OHIO STATE', False),
 (u'WROTE A LETTER TO THE PRESIDENT OF THE UNITED STATES BASICALLY INVITING MYSELF DOWN THERE FOR A CHAT IT',
  False),
 (u'CARRIED BY THE PRESIDENT OF OHIO STATE WHO I HAD MET JUST A SHORT TIME EARLIER HE',
  False),
 (u'ME BACK AND INVITED ME TO A', False),
 (u'MEETING OFFICE I REMEMBER CALLING HOME AND MY MOTHER', False),
 (u"THE PHONE AND I SAID MOM I'M A FIRST QUARTER FRESHMAN AT OHIO STATE MY",
  False),
 (u'CARRIED MAIL AND', False),
 (u'SAID I NEED AN AIRLINE TICKET BECAUSE THE PRESIDENT THE UNITED TO HAVE A MEETING THE OVAL OFFICE',
  False),
 (u"MOM WAS SHOUTING TO THE PHONE THERE'S SOMETHING WRONG WITH MY KID I'VE HAD A COUPLE",
  False),
 (u'IN WASHINGTON', False),
 (u'CONTINUED TO STRIKE ME', False),
 (u"WHEN I GRADUATED FROM OHIO STATE I WENT LOOKING FOR A JOB AND COULDN'T FIND ONE THE LAST PLACE THOUGHT I WOULD BE HIRED I

In [157]:
jeb_0_activations = zip(np.array(all_feature_names)[jeb_order_0], jeb_train[0]*coefs)
jeb_1_activations = zip(np.array(all_feature_names)[jeb_order_1], jeb_train[1]*coefs[jeb_order_1])
jeb_2_activations = zip(np.array(all_feature_names)[jeb_order_2], jeb_train[2]*coefs[jeb_order_2])

In [160]:
jeb_1_activations[-50:]

[(u'be_a', 0.0),
 (u'LIWC_friend', 0.0),
 (u'min_phone_length', -0.11841505438139058),
 (u'LIWC_percept', 0.0),
 (u'the_business', 0.0),
 (u'LIWC_we', 0.0),
 (u'mean_pitch', -0.0),
 (u'back_in', 0.0),
 (u'internal_silence', 0.1178684567214011),
 (u'LIWC_power', 0.0),
 (u'will_be', 0.058892930007002173),
 (u'LIWC_i', 0.0),
 (u'of_this', 0.0),
 (u'country_to', 0.0),
 (u'rhyme', 0.0),
 (u'LIWC_auxverb', 0.0),
 (u'LIWC_focuspresent', 0.0),
 (u'a_commander', 0.0),
 (u'LIWC_motion', 0.077162140595016776),
 (u'in_the', 0.0),
 (u'LIWC_negate', 0.0),
 (u'rst_cat_N-purpose', 0.090784796912178556),
 (u'LIWC_achiev', 0.0),
 (u"i_won't", 0.0),
 (u'LIWC_female', 0.0),
 (u'the_military', 0.0),
 (u'get_back', 0.0),
 (u'LIWC_affiliation', 0.0),
 (u'out_there', 0.0),
 (u'a_lot', 0.11195111771020727),
 (u'will_have', 0.0),
 (u'to_act', 0.0),
 (u'LIWC_sexual', 0.0),
 (u'alliteration', 0.0),
 (u'who_will', 0.0),
 (u'LIWC_leisure', 0.0),
 (u'i_think', 0.0),
 (u'LIWC_reward', 0.0),
 (u'next_president', 0.0),

In [149]:
jeb_fails_features[-50:]

[(u'be_a', 0.031838287462172883),
 (u'LIWC_friend', 0.033482519190737206),
 (u'min_phone_length', -0.11841505438139058),
 (u'LIWC_percept', 0.037158682089851985),
 (u'the_business', 0.047011592795904479),
 (u'LIWC_we', 0.047574463120682461),
 (u'mean_pitch', -0.081377825397440734),
 (u'back_in', 0.049425862091193665),
 (u'internal_silence', 0.1178684567214011),
 (u'LIWC_power', 0.05678676794291547),
 (u'will_be', 0.058892930007002173),
 (u'LIWC_i', 0.061607090604034916),
 (u'of_this', 0.066118407633243184),
 (u'country_to', 0.067722884021763075),
 (u'rhyme', 0.019610660860075655),
 (u'LIWC_auxverb', 0.072545241142137784),
 (u'LIWC_focuspresent', 0.075311245702706392),
 (u'a_commander', 0.076039724090637098),
 (u'LIWC_motion', 0.077162140595016776),
 (u'in_the', 0.07748078562768182),
 (u'LIWC_negate', 0.087615537447223946),
 (u'rst_cat_N-purpose', 0.090784796912178556),
 (u'LIWC_achiev', 0.092107404498263407),
 (u"i_won't", 0.094331942203364064),
 (u'LIWC_female', 0.097161055641983796),

In [138]:
zip(jeb.alignment.get_phrase_text(), model.predict_proba(jeb_train)[:,1])

[(u"So here's my pledge to you", 0.24821266037726158),
 (u"will be a commander in chief who will have the back of the military I won't trash talk I won't be a divider in chief or an agitator in chief I won't be out there blowharding talking a big game without backing it up I think the next President needs to be a lot quieter but send a signal that we're prepared to act in the national security interests of this country to get back in the business of creating a more peaceful world",
  0.9450858130262213),
 (u'Please clap', 0.39974463627379864)]

In [73]:
feat = featurizer.Featurizer(s)
#feature_set = ['audio','liu','euphony','phone','cosine','substring','word_overlap','rst']

In [74]:
def make_delta(feature_names):
    return ["delta_"+f for f in feature_names]

In [78]:
all_feature_names = feat.get_audio_feature_names() + make_delta(feat.get_audio_feature_names()) + feat.get_liu_feature_names() + make_delta(feat.get_liu_feature_names()) + feat.get_euphony_feature_names() + make_delta(feat.get_euphony_feature_names()) + feat.get_phone_feature_names() + make_delta(feat.get_phone_feature_names())+feat.get_cosine_feature_names() + make_delta(feat.get_cosine_feature_names())+feat.get_substring_feature_names() + make_delta(feat.get_substring_feature_names())+ feat.get_word_overlap_feature_names() + make_delta(feat.get_word_overlap_feature_names())+feat.get_rst_feature_names() + make_delta(feat.get_rst_feature_names())

In [92]:
all_feature_names = all_bigram_names + feat.get_audio_feature_names()+ feat.get_liu_feature_names() + feat.get_euphony_feature_names() + feat.get_phone_feature_names() + feat.get_cosine_feature_names() +feat.get_substring_feature_names() + feat.get_word_overlap_feature_names() +feat.get_rst_feature_names() 

In [80]:
with open('/data/corpora/cspan/top_bigrams.pkl','rb') as f:
    top_bigrams = pickle.load(f)

In [83]:
all_bigram_names = [n[0] for n in top_bigrams]

In [85]:
all_bigram_names.sort()

In [87]:
all_bigram_names[0:10]

[u'000_50',
 u'000_70',
 u'000_a',
 u'000_e',
 u'000_factories',
 u'000_in',
 u'000_jobs',
 u'000_or',
 u'000_people',
 u'000_soldiers']

In [95]:
coef_list = zip(np.array(all_feature_names)[np.argsort(coefs)],coefs[np.argsort(coefs)])

In [102]:
with open('/data/jrgillick/coef_list.txt','wb') as f:
    for c in coef_list:
        f.write(str(c[0]) + ' ' + str(c[1]) + '\n')

In [103]:
speechjeb 

<module 'speech' from 'speech.pyc'>

In [99]:
str(coef_list[0][0]) + ' ' + str(coef_list[0][1])

'talk_about -0.566926635654'

In [51]:
s = speech.Speech('bernie_sanders_10')

In [30]:
def run_logistic_cross_validation(feature_set, phrase_count):
    all_true = []; all_pred = []
    C = 0.1
    for speaker in speaker_names:
        print speaker + ": "
        train_set, test_set = get_train_and_test_sets(speaker)

        labels_train = get_labels(train_set)
        data_train = get_features(train_set,feature_set)

        labels_test = get_labels(test_set)
        data_test = get_features(test_set,feature_set)

        current_train_formatted_feats, current_train_formatted_labs = models.format_balanced_multiple_phrase_input(data_train,labels_train,phrase_count=phrase_count)
        current_test_formatted_feats, current_test_formatted_labs = models.format_balanced_multiple_phrase_input(data_test,labels_test,phrase_count=phrase_count)

        X_train = np.array(current_train_formatted_feats).astype('float64')
        y_train = np.array(current_train_formatted_labs)
        
        print X_train.shape

        X_test = np.array(current_test_formatted_feats).astype('float64')
        y_test = np.array(current_test_formatted_labs)

        X_train, X_test = normalize_data(X_train,X_test)
        if C is None:
            model = models.train_cv_logistic_regression_faster(X_train,y_train)
            C = model.best_params_['C']
        else:
            model = models.train_logistic_regression(X_train,y_train,C)
        y_true, y_pred = models.evaluate_model(model, X_test, y_test)
        
        all_true += list(y_true)
        all_pred += list(y_pred)
        print
    return all_true, all_pred

In [32]:
def run_logistic_single_speaker(feature_set, phrase_count):
    all_true = []; all_pred = []
    C = None
    for speaker in speaker_names:
        print speaker + ": "
        other, speaker_data = get_train_and_test_sets(speaker)
        speaker_data = np.array(speaker_data)
        #speaker_labels = get_labels(speaker_data)
        
        if len(speaker_data) == 1:
            continue
        
        if len(speaker_data) > 10:
            kf = KFold(n_splits=10)
        else:
            kf = KFold(n_splits=len(speaker_data))
            
        for train_index, test_index in kf.split(speaker_data):
            train_set = speaker_data[train_index]
            test_set = speaker_data[test_index]
            #train_set, test_set = train_test_split(speaker_data, test_size=0.2, random_state=43)

            labels_train = get_labels(train_set)
            data_train = get_features(train_set,feature_set)

            labels_test = get_labels(test_set)
            data_test = get_features(test_set,feature_set)

            current_train_formatted_feats, current_train_formatted_labs = models.format_balanced_multiple_phrase_input(data_train,labels_train,phrase_count=phrase_count)
            current_test_formatted_feats, current_test_formatted_labs = models.format_balanced_multiple_phrase_input(data_test,labels_test,phrase_count=phrase_count)

            X_train = np.array(current_train_formatted_feats).astype('float64')
            y_train = np.array(current_train_formatted_labs)

            X_test = np.array(current_test_formatted_feats).astype('float64')
            y_test = np.array(current_test_formatted_labs)

            #print X_train.shape
            #print X_test.shape

            if len(y_train) > 0 and len(y_test) > 0:

                X_train, X_test = normalize_data(X_train,X_test)
                if C is None and len(y_train) > 500:
                    model = models.train_cv_logistic_regression_faster(X_train,y_train)
                    C = model.best_params_['C']
                else:
                    model = models.train_logistic_regression(X_train,y_train)
                y_true, y_pred = models.evaluate_model(model, X_test, y_test)
                
                all_true += list(y_true)
                all_pred += list(y_pred)
                
                #print len(all_true)
            print
    #print len(all_true)
    #print len(all_pred)
    return all_true, all_pred

In [24]:
def run_logistic_single_speaker_n_gram_and_deltas(feature_set, phrase_count):
    feature_set.remove('bigrams')
    all_true = []; all_pred = []
    C = None
    for speaker in speaker_names:
        print speaker + ": "
        other, speaker_data = get_train_and_test_sets(speaker)
        speaker_data = np.array(speaker_data)
        #speaker_labels = get_labels(speaker_data)
        
        if len(speaker_data) == 1:
            continue
        
        if len(speaker_data) > 10:
            kf = KFold(n_splits=10)
        else:
            kf = KFold(n_splits=len(speaker_data))
            
        for train_index, test_index in kf.split(speaker_data):
            train_set = speaker_data[train_index]
            test_set = speaker_data[test_index]
            #train_set, test_set = train_test_split(speaker_data, test_size=0.2, random_state=43)

            labels_train = get_labels(train_set)
            data_train = get_features(train_set,feature_set)
            
            n_grams_train = get_features(train_set,['bigrams'])

            labels_test = get_labels(test_set)
            data_test = get_features(test_set,feature_set)
            n_grams_test = get_features(test_set,['bigrams'])

            current_train_formatted_feats, current_train_formatted_labs = models.format_balanced_multiple_phrase_input_with_deltas(data_train,labels_train,phrase_count=phrase_count)
            current_test_formatted_feats, current_test_formatted_labs = models.format_balanced_multiple_phrase_input_with_deltas(data_test,labels_test,phrase_count=phrase_count)

            train_ngram_formatted_feats, current_train_formatted_labs = models.format_balanced_multiple_phrase_input(n_grams_train,labels_train,phrase_count=1)
            test_ngram_formatted_feats, current_test_formatted_labs = models.format_balanced_multiple_phrase_input(n_grams_test,labels_test,phrase_count=1)

            
            X_train = np.array(current_train_formatted_feats).astype('float64')
            y_train = np.array(current_train_formatted_labs)
            
            X_train_2 = np.array(train_ngram_formatted_feats).astype('float64')

            X_test = np.array(current_test_formatted_feats).astype('float64')
            y_test = np.array(current_test_formatted_labs)
            
            X_test_2 = np.array(test_ngram_formatted_feats).astype('float64')

            print X_train.shape
            print X_train_2.shape
            print X_test.shape
            print X_test_2.shape
            
            X_train = np.hstack([X_train,X_train_2])
            X_test = np.hstack([X_test,X_test_2])

            if len(y_train) > 0 and len(y_test) > 0:

                X_train, X_test = normalize_data(X_train,X_test)
                if C is None and len(y_train) > 500:
                    model = models.train_cv_logistic_regression_faster(X_train,y_train)
                    C = model.best_params_['C']
                else:
                    model = models.train_logistic_regression(X_train,y_train)
                y_true, y_pred = models.evaluate_model(model, X_test, y_test)
                
                all_true += list(y_true)
                all_pred += list(y_pred)
                
                #print len(all_true)
            print
    #print len(all_true)
    #print len(all_pred)
    return all_true, all_pred

In [34]:
#all_true, all_pred = run_logistic_single_speaker_n_gram_and_deltas(['bigrams','audio','liu','euphony','phone','cosine','substring','word_overlap','rst'],phrase_count=1)

In [307]:
all_true,all_pred = run_logistic_cross_validation(['bigrams'],phrase_count=1)

donald_trump: 
(22678, 26704)
Accuracy: 0.575 +/- 0.009 (7387/12846) | Precision: 0.626 | Recall: 0.373 | F1: 0.467

hilary_clinton: 
(28264, 26704)
Accuracy: 0.612 +/- 0.011 (4440/7260) | Precision: 0.645 | Recall: 0.497 | F1: 0.561

bernie_sanders: 
(29006, 26704)
Accuracy: 0.628 +/- 0.012 (4092/6518) | Precision: 0.677 | Recall: 0.489 | F1: 0.568

ted_cruz: 
(33442, 26704)
Accuracy: 0.626 +/- 0.021 (1304/2082) | Precision: 0.684 | Recall: 0.469 | F1: 0.556

marco_rubio: 
(33938, 26704)
Accuracy: 0.627 +/- 0.024 (994/1586) | Precision: 0.658 | Recall: 0.528 | F1: 0.586

john_kasich: 
(34886, 26704)
Accuracy: 0.630 +/- 0.037 (402/638) | Precision: 0.706 | Recall: 0.445 | F1: 0.546

barack_obama: 
(33684, 26704)
Accuracy: 0.559 +/- 0.023 (1028/1840) | Precision: 0.624 | Recall: 0.296 | F1: 0.401

bill_clinton: 
(34940, 26704)
Accuracy: 0.596 +/- 0.040 (348/584) | Precision: 0.665 | Recall: 0.387 | F1: 0.489

joe_biden: 
(34984, 26704)
Accuracy: 0.615 +/- 0.041 (332/540) | Precision: 0.

In [309]:
total_correct_n_gram = 7387 + 4440 + 4092 + 1304 + 994 + 402 + 1028 + 348 + 332 + 314 + 163 + 174+58+48+19
over_n_gram = 12846 + 7260 + 6518 + 2082 + 1586 + 638 + 1840 + 584 + 540 + 492 + 258 + 382 + 268 + 112 + 84 + 34
acc_here = float(total_correct_n_gram) / over_n_gram

In [310]:
acc_here

0.5940490935705438

In [305]:
all_true,all_pred = run_logistic_single_speaker(['bigrams','audio','liu','euphony','phone','cosine','substring','word_overlap','rst'],phrase_count=1)

donald_trump: 
Accuracy: 0.650 +/- 0.022 (1210/1862) | Precision: 0.674 | Recall: 0.581 | F1: 0.624

Accuracy: 0.664 +/- 0.028 (752/1132) | Precision: 0.689 | Recall: 0.599 | F1: 0.641

Accuracy: 0.674 +/- 0.027 (781/1158) | Precision: 0.705 | Recall: 0.599 | F1: 0.648

Accuracy: 0.664 +/- 0.027 (756/1138) | Precision: 0.679 | Recall: 0.622 | F1: 0.650

Accuracy: 0.676 +/- 0.022 (1143/1692) | Precision: 0.684 | Recall: 0.651 | F1: 0.667

Accuracy: 0.640 +/- 0.023 (1108/1730) | Precision: 0.664 | Recall: 0.568 | F1: 0.612

Accuracy: 0.657 +/- 0.027 (796/1212) | Precision: 0.666 | Recall: 0.630 | F1: 0.647

Accuracy: 0.688 +/- 0.033 (523/760) | Precision: 0.698 | Recall: 0.663 | F1: 0.680

Accuracy: 0.642 +/- 0.026 (860/1340) | Precision: 0.649 | Recall: 0.618 | F1: 0.633

Accuracy: 0.676 +/- 0.032 (556/822) | Precision: 0.693 | Recall: 0.633 | F1: 0.662

hilary_clinton: 
Accuracy: 0.689 +/- 0.033 (528/766) | Precision: 0.677 | Recall: 0.723 | F1: 0.699

Accuracy: 0.700 +/- 0.035 (466/66

In [ ]:
trump_correct = 1210+752+781+756+1143+1108+796+523+860+556
trump_total = 1862+1132+1158+1138+1692+1730+1212+760+1340+822

In [33]:
all_true,all_pred = run_logistic_cross_validation(['audio','liu','euphony','phone','cosine','substring','word_overlap','rst'],phrase_count=1)

donald_trump: 
(22678, 155)
Accuracy: 0.588 +/- 0.009 (7559/12846) | Precision: 0.600 | Recall: 0.531 | F1: 0.563

hilary_clinton: 
(28264, 155)
Accuracy: 0.658 +/- 0.011 (4774/7260) | Precision: 0.653 | Recall: 0.674 | F1: 0.663

bernie_sanders: 
(29006, 155)
Accuracy: 0.616 +/- 0.012 (4017/6518) | Precision: 0.637 | Recall: 0.542 | F1: 0.586

ted_cruz: 
(33442, 155)
Accuracy: 0.632 +/- 0.021 (1315/2082) | Precision: 0.659 | Recall: 0.545 | F1: 0.597

marco_rubio: 
(33938, 155)
Accuracy: 0.637 +/- 0.024 (1011/1586) | Precision: 0.639 | Recall: 0.631 | F1: 0.635

john_kasich: 
(34886, 155)
Accuracy: 0.672 +/- 0.036 (429/638) | Precision: 0.688 | Recall: 0.630 | F1: 0.658

barack_obama: 
(33684, 155)
Accuracy: 0.637 +/- 0.022 (1173/1840) | Precision: 0.655 | Recall: 0.582 | F1: 0.616

bill_clinton: 
(34940, 155)
Accuracy: 0.625 +/- 0.039 (365/584) | Precision: 0.629 | Recall: 0.610 | F1: 0.619

joe_biden: 
(34984, 155)
Accuracy: 0.617 +/- 0.041 (333/540) | Precision: 0.618 | Recall: 0.6

In [308]:
#total_correct = np.sum(np.array(all_true) == np.array(all_pred))
acc=float(total_correct) / len(all_true)
std=math.sqrt( (acc * (1-acc)) / len(all_true) )
precision, recall, f1, support = [l[1] for l in precision_recall_fscore_support(all_true, all_pred)]
print "Accuracy: %.3f +/- %.3f (%s/%s) | Precision: %.3f | Recall: %.3f | F1: %.3f" % (acc, 1.96*std, total_correct, len(all_true), precision, recall, f1)

Accuracy: 0.668 +/- 0.005 (23740/35524) | Precision: 0.650 | Recall: 0.435 | F1: 0.521


In [29]:
total_correct = 1171 + 748 + 767 + 756 + 1135 + 1101 + 794 + 522 + 868 + 571+ 528 + 465 + 381 + 506 + 563 + 539 + 741 + 497 + 470 + 421 + 584 + 341 + 294 + 615 + 321 + 619 + 249 + 475 + 571+ 586 + 178 + 118 + 195+135 +81 + 126 + 154 + 127 + 174 + 95+130 + 115+ 84 + 93 + 174 + 39 + 118 + 118 + 51 + 69 + 31 + 49 + 58 + 51 + 59 + 78 + 33 + 11 + 15 + 35 + 98 + 245 + 81 + 161 + 197 + 77 + 110 + 129 + 21 + 57 + 30 + 28 + 13 + 116 + 40 + 11 + 91 + 29 + 32 + 115+ 60 + 25 + 84 + 50 + 71 + 52+ 5+57 + 65 + 24 + 25 + 34 + 40 + 9 + 124 + 44 + 15 + 34 + 4 + 31 + 33 + 1 + 75 + 16 + 8 + 37 + 14 + 14 + 15
total_predicted = 1862 + 1132 + 1158 + 1138 + 1692 + 1730 + 1212 + 760 + 1340 + 822 + 766+ 666+536 + 744+780 +556 + 1122 + 760 + 694 + 636 + 824 + 466 + 406 + 884 + 466 + 882 + 348 + 626 + 794 + 822 + 260+ 182+316+208 + 114 + 180+228+188+276+130 + 236 + 196 + 136 + 142 + 274 + 54 + 188 + 166 + 72 + 122 + 40 + 82 + 84 + 74 + 88 + 108 + 52 + 26 + 20 + 64 + 152 + 392 + 116 + 222 + 328 + 138 + 170 + 196 + 34+92 + 56 + 48 + 22 + 190 + 62 + 18 + 140 + 48 + 54 + 212 + 92 + 60 + 122+80 + 126 + 90 + 8 + 84 + 104 + 40 + 58 + 62 + 78 + 20 + 200 + 76 + 32 + 64 + 10 + 62 + 62+2+142+26+16+70+20+34+30
float(total_correct)/total_predicted

0.6689208227669766

In [13]:
import math

In [1150]:
total_words = 0
aligned_words = 0
for s in speeches:
    for word in s.alignment.words:
        total_words += 1
        if word['case'] == 'success':
            aligned_words += 1

In [1152]:
float(aligned_words)/total_words

0.8783685419891444

In [1]:
total_words

NameError: name 'total_words' is not defined

In [1155]:
aligned_words

1077608

In [1143]:
s.alignment.words[55]

{u'case': u'not-found-in-audio',
 u'endOffset': 313,
 u'startOffset': 311,
 u'word': u'IN'}

In [1144]:
s.alignment.words[54]

{u'alignedWord': u'people',
 u'case': u'success',
 u'end': 2233.5099999999998,
 u'endOffset': 310,
 u'phones': [{u'duration': 0.07, u'phone': u'p_B'},
  {u'duration': 0.09, u'phone': u'iy_I'},
  {u'duration': 0.04, u'phone': u'p_I'},
  {u'duration': 0.06, u'phone': u'ah_I'},
  {u'duration': 0.03, u'phone': u'l_E'}],
 u'start': 2233.22,
 u'startOffset': 304,
 u'word': u'PEOPLE'}

In [1027]:
all_true, all_pred = run_logistic_cross_validation(['audio','liu','euphony','phone','cosine','substring','word_overlap','rst'],phrase_count=1)

donald_trump: 
(22678, 310)
Accuracy: 0.587 +/- 0.009 (7535/12846) | Precision: 0.596 | Recall: 0.536 | F1: 0.564

hilary_clinton: 
(28264, 310)
Accuracy: 0.659 +/- 0.011 (4783/7260) | Precision: 0.647 | Recall: 0.697 | F1: 0.671

bernie_sanders: 
(29006, 310)
Accuracy: 0.610 +/- 0.012 (3975/6518) | Precision: 0.632 | Recall: 0.525 | F1: 0.574

ted_cruz: 
(33442, 310)
Accuracy: 0.614 +/- 0.021 (1279/2082) | Precision: 0.637 | Recall: 0.530 | F1: 0.579

marco_rubio: 
(33938, 310)
Accuracy: 0.626 +/- 0.024 (993/1586) | Precision: 0.627 | Recall: 0.622 | F1: 0.624

john_kasich: 
(34886, 310)
Accuracy: 0.688 +/- 0.036 (439/638) | Precision: 0.699 | Recall: 0.661 | F1: 0.680

barack_obama: 
(33684, 310)
Accuracy: 0.620 +/- 0.022 (1141/1840) | Precision: 0.639 | Recall: 0.553 | F1: 0.593

bill_clinton: 
(34940, 310)
Accuracy: 0.618 +/- 0.039 (361/584) | Precision: 0.618 | Recall: 0.620 | F1: 0.619

joe_biden: 
(34984, 310)
Accuracy: 0.639 +/- 0.041 (345/540) | Precision: 0.652 | Recall: 0.59

In [1070]:
total_correct = np.sum(np.array(all_true) == np.array(all_pred))
acc=float(total_correct) / len(all_true)
std=math.sqrt( (acc * (1-acc)) / len(all_true) )
precision, recall, f1, support = [l[1] for l in precision_recall_fscore_support(all_true, all_pred)]
print "Accuracy: %.3f +/- %.3f (%s/%s) | Precision: %.3f | Recall: %.3f | F1: %.3f" % (acc, 1.96*std, total_correct, len(all_true), precision, recall, f1)

Accuracy: 0.609 +/- 0.011 (4230/6942) | Precision: 0.614 | Recall: 0.590 | F1: 0.602


In [1075]:
len(all_pred)

6942

In [ ]:
#1 phrase, with deltas ['audio','liu','euphony','phone','cosine','substring','word_overlap','rst']
#Accuracy: 0.617 +/- 0.005 (21907/35524) | Precision: 0.625 | Recall: 0.582 | F1: 0.603

#1 phrase, no deltas ['audio','liu','euphony','phone','cosine','substring','word_overlap','rst']
#Accuracy: 0.616 +/- 0.005 (21871/35524) | Precision: 0.626 | Recall: 0.576 | F1: 0.600

#1 phrase, no deltas  ['audio','liu','euphony','phone']
#Accuracy: 0.615 +/- 0.005 (21852/35524) | Precision: 0.624 | Recall: 0.579 | F1: 0.601

#3 phrases, no deltas  ['audio','liu','euphony','phone']
#Accuracy: 0.618 +/- 0.005 (21684/35092) | Precision: 0.627 | Recall: 0.582 | F1: 0.604



In [993]:
total_correct

21569

In [692]:
from sklearn.metrics import precision_recall_fscore_support

In [1009]:
reload(models)

<module 'models' from 'models.py'>

In [805]:
type(np.array(test_rst_feats2)[0])

list

In [806]:
type(np.array(test_rst_feats)[0])

list

In [1004]:
model.best_params_['C']

0.01

In [997]:
reload(models)

<module 'models' from 'models.py'>

Accuracy: 0.551 +/- 0.009 (7079/12846) | Precision: 0.562 | Recall: 0.461 | F1: 0.507


(7079, 12846)

In [652]:
len(X_test)

12846

In [656]:
reload(models)

<module 'models' from 'models.py'>

In [ ]:
#train_rst_feats = data_formatter.get_rst_data(train_speeches)
#train_audio_feats = data_formatter.get_audio_data(train_speeches)
#train_labels = data_formatter.get_binary_labels(train_speeches)
#train_euphony_feats = data_formatter.get_euphony_data(train_speeches)
#train_liu_feats = data_formatter.get_liu_data(train_speeches)
#train_cosine_feats = data_formatter.get_vector_cosine_distance_data(train_speeches)
#train_phone_feats = data_formatter.get_phone_data(train_speeches)
#train_substring_feats = data_formatter.get_substring_data(train_speeches)
#train_word_overlap_feats = data_formatter.get_word_overlap_data(train_speeches)

In [699]:
train_rst_feats2 = data_formatter.get_rst_data(train_speeches)

100%|██████████| 248/248 [00:01<00:00, 148.27it/s]


In [700]:
test_rst_feats2 = data_formatter.get_rst_data(test_speeches)

100%|██████████| 62/62 [00:00<00:00, 128.87it/s]


In [737]:
len(train_rst_feats2[10])

259

In [738]:
len(train_rst_feats[10])

259

In [718]:
len(test_rst_feats[-1][0])

59

In [430]:
np.array(train_rst_feats[0]).shape

(107, 59)

In [457]:
test_bigram_feats = data_formatter.get_bigram_data(test_speeches[0:5])

100%|██████████| 5/5 [01:04<00:00, 14.48s/it]


In [462]:
test_bigram_feats[0].shape

(244, 26704)

In [454]:
np.nonzero(test_bigram_feats[0])

(array([  0,   0,   0, ..., 242, 243, 243]),
 array([2704, 3220, 4088, ..., 6587, 1226, 3232]))

In [434]:
test_unigram_feats[0].shape

(244, 4)

In [438]:
np.array(test_bigram_feats[0]).shape

(244,)

In [441]:
text_features.TextFeatures('this is a phrase').get_n_grams(2)

['this_is', 'is_a', 'a_phrase']

In [343]:
#train_unigram_feats = data_formatter.get_unigram_data(train_speeches)
train_rst_feats = data_formatter.get_rst_data(train_speeches)
train_audio_feats = data_formatter.get_audio_data(train_speeches)
train_labels = data_formatter.get_binary_labels(train_speeches)
train_euphony_feats = data_formatter.get_euphony_data(train_speeches)
train_liu_feats = data_formatter.get_liu_data(train_speeches)
train_cosine_feats = data_formatter.get_vector_cosine_distance_data(train_speeches)
train_phone_feats = data_formatter.get_phone_data(train_speeches)
train_substring_feats = data_formatter.get_substring_data(train_speeches)
train_word_overlap_feats = data_formatter.get_word_overlap_data(train_speeches)

100%|██████████| 248/248 [00:00<00:00, 281.08it/s]


In [344]:
#test_unigram_feats = data_formatter.get_unigram_data(test_speeches)
test_rst_feats = data_formatter.get_rst_data(test_speeches)
test_audio_feats = data_formatter.get_audio_data(test_speeches)
test_labels = data_formatter.get_binary_labels(test_speeches)
test_euphony_feats = data_formatter.get_euphony_data(test_speeches)
test_liu_feats = data_formatter.get_liu_data(test_speeches)
test_cosine_feats = data_formatter.get_vector_cosine_distance_data(test_speeches)
test_phone_feats = data_formatter.get_phone_data(test_speeches)
test_substring_feats = data_formatter.get_substring_data(test_speeches)
test_word_overlap_feats = data_formatter.get_word_overlap_data(test_speeches)

100%|██████████| 62/62 [00:00<00:00, 287.97it/s]


In [427]:
train_euphony_feats[0].shape

(107, 4)

In [428]:
train_unigram_feats[0].shape

(453, 4259)

In [426]:
all_train_t0_feats =  [np.hstack([train_unigram_feats[i],train_audio_feats[i],train_euphony_feats[i],train_liu_feats[i],train_phone_feats[i]]) for i in range(len(train_audio_feats))]
all_test_t0_feats =  [np.hstack([test_unigram_feats[i],test_audio_feats[i],test_euphony_feats[i],test_liu_feats[i],test_phone_feats[i]]) for i in range(len(test_audio_feats))]

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [537]:
len(train_labels[0])

107

In [345]:
all_train_feats = [np.hstack([train_rst_feats[i],train_audio_feats[i],train_euphony_feats[i],train_liu_feats[i],train_phone_feats[i],train_substring_feats[i].reshape(-1,1),train_word_overlap_feats[i].reshape(-1,1),np.array(train_cosine_feats[i]).reshape(-1,1)]) for i in range(len(train_audio_feats))]
all_test_feats = [np.hstack([test_rst_feats[i],test_audio_feats[i],test_euphony_feats[i],test_liu_feats[i],test_phone_feats[i],test_substring_feats[i].reshape(-1,1),test_word_overlap_feats[i].reshape(-1,1),np.array(test_cosine_feats[i]).reshape(-1,1)]) for i in range(len(test_audio_feats))]

In [326]:
all_train_rst_feats = [np.hstack([train_rst_feats[i]]) for i in range(len(train_audio_feats))]
all_test_rst_feats = [np.hstack([test_rst_feats[i]]) for i in range(len(test_audio_feats))]

In [340]:
train_rst_formatted_feats, train_rst_formatted_labs = models.format_balanced_multiple_phrase_input(all_train_rst_feats,train_labels,phrase_count=1)
test_rst_formatted_feats, test_rst_formatted_labs = models.format_balanced_multiple_phrase_input(all_test_rst_feats,test_labels,phrase_count=1)

Empty...18
Empty...28
Empty...36
Empty...157


In [341]:
X_train, y_train = np.array(train_rst_formatted_feats), np.array(train_rst_formatted_labs)
X_test, y_test = np.array(test_rst_formatted_feats), np.array(test_rst_formatted_labs)

In [158]:
#train_formatted_feats_t0, train_formatted_labs_t0 = models.format_multiple_phrase_input(all_train_t0_feats,train_labels,phrase_count=1)
#test_formatted_feats_t0, test_formatted_labs_t0 = models.format_multiple_phrase_input(all_test_t0_feats,test_labels,phrase_count=1)

In [335]:
type(train_rst_feats[0])

list

In [336]:
featurizer.Featurizer(s).get_rst_feature_names()

['rst_cat_N-antithesis',
 'rst_cat_N-attribution',
 'rst_cat_N-circumstance',
 'rst_cat_N-comment',
 'rst_cat_N-comparison',
 'rst_cat_N-concession',
 'rst_cat_N-condition',
 'rst_cat_N-consequence',
 'rst_cat_N-contrast',
 'rst_cat_N-definition',
 'rst_cat_N-disjunction',
 'rst_cat_N-elaboration',
 'rst_cat_N-evidence',
 'rst_cat_N-example',
 'rst_cat_N-explanation',
 'rst_cat_N-hypothetical',
 'rst_cat_N-list',
 'rst_cat_N-manner',
 'rst_cat_N-means',
 'rst_cat_N-purpose',
 'rst_cat_N-question',
 'rst_cat_N-reason',
 'rst_cat_N-restatement',
 'rst_cat_N-result',
 'rst_cat_N-same_unit',
 'rst_cat_N-sequence',
 'rst_cat_N-temporal',
 'rst_cat_N-textualorganization',
 'rst_cat_N-topic',
 'rst_cat_S-antithesis',
 'rst_cat_S-attribution',
 'rst_cat_S-circumstance',
 'rst_cat_S-comment',
 'rst_cat_S-comparison',
 'rst_cat_S-concession',
 'rst_cat_S-condition',
 'rst_cat_S-consequence',
 'rst_cat_S-definition',
 'rst_cat_S-elaboration',
 'rst_cat_S-example',
 'rst_cat_S-explanation',
 'rst_

In [8]:
#unigram_list = featurizer.Featurizer(s).get_unigram_features()

In [9]:
#sparse.dok_matrix(np.array(unigram_list).shape)

In [346]:
train_formatted_feats, train_formatted_labs = models.format_balanced_multiple_phrase_input_with_deltas(all_train_feats,train_labels,phrase_count=5)
test_formatted_feats, test_formatted_labs = models.format_balanced_multiple_phrase_input_with_deltas(all_test_feats,test_labels,phrase_count=5)

Empty...18
Empty...28
Empty...36
Empty...157


In [347]:
X_train, y_train = np.array(train_formatted_feats), np.array(train_formatted_labs)
X_test, y_test = np.array(test_formatted_feats), np.array(test_formatted_labs)

In [10]:
X_train, y_train = np.array(train_formatted_feats_t0), np.array(train_formatted_labs_t0)
X_test, y_test = np.array(test_formatted_feats_t0), np.array(test_formatted_labs_t0)

In [167]:
X_train.shape

(28132, 1550)

In [168]:
X_test.shape

(6514, 1550)

In [348]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [349]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [350]:
with open('/data/jrgillick/x_train.pkl','wb') as f:
    pickle.dump(X_train,f)
    
with open('/data/jrgillick/x_test.pkl','wb') as f:
    pickle.dump(X_test,f)

with open('/data/jrgillick/y_train.pkl','wb') as f:
    pickle.dump(y_train,f)
    
with open('/data/jrgillick/y_test.pkl','wb') as f:
    pickle.dump(y_test,f)

In [352]:
s = speeches[221]
s.file_path
s.phrase_audio_features[19]

{'internal_silence': 0.3643679755792979,
 'max_energy': 0.27192634,
 'max_pitch': 172.26562,
 'mean_energy': 0.084792912,
 'mean_pitch': 44.370567,
 'min_energy': 0.0015051069,
 'min_pitch': -1.0,
 'range_energy': 0.27042124,
 'range_pitch': 173.26562,
 'std_energy': 0.057384759,
 'std_pitch': 42.044147}

In [356]:
file_paths = [s.file_path.split('/')[0] for s in speeches]
counts = {}
for p in file_paths:
    if p in counts:
        counts[p] +=1 
    else:
        counts[p] = 1

In [362]:
keys = counts.keys()
values = counts.values()
order = np.argsort(values)

In [474]:
l = zip(list(np.array(keys)[order]),list(np.array(values)[order]))
l.reverse()
l

[('donald_trump', 86),
 ('hilary_clinton', 72),
 ('bernie_sanders', 40),
 ('ted_cruz', 23),
 ('marco_rubio', 20),
 ('john_kasich', 17),
 ('barack_obama', 10),
 ('bill_clinton', 8),
 ('joe_biden', 7),
 ('mike_pence', 6),
 ('carly_fiorina', 5),
 ('jeb_bush', 5),
 ('rand_paul', 4),
 ('gary_johnson', 3),
 ('chris_christie', 3),
 ('rick_santorum', 1)]

In [479]:
speaker_names = [i[0] for i in l]
speaker_names

['donald_trump',
 'hilary_clinton',
 'bernie_sanders',
 'ted_cruz',
 'marco_rubio',
 'john_kasich',
 'barack_obama',
 'bill_clinton',
 'joe_biden',
 'mike_pence',
 'carly_fiorina',
 'jeb_bush',
 'rand_paul',
 'gary_johnson',
 'chris_christie',
 'rick_santorum']

In [387]:
total_u = 0
total_c = 0
for k in keys:
    k_speeches = [s for s in speeches if k in s.file_path]
    num_utterances = 0
    num_claps = 0
    for s in k_speeches:
        times = s.alignment.get_phrase_times()
        binary_labels = [int(s.alignment.applause_follows(t[1])) for t in times]
        num_utterances += (len(times))
        num_claps += np.sum(binary_labels)
    print k + " %d %d %f" % (num_utterances, num_claps, float(num_claps)/num_utterances)
    total_u += num_utterances
    total_c += num_claps

jeb_bush 1482 191 0.128880
joe_biden 1847 270 0.146183
hilary_clinton 12825 3933 0.306667
bernie_sanders 10994 3529 0.320993
rand_paul 939 134 0.142705
carly_fiorina 1222 129 0.105565
bill_clinton 2087 292 0.139914
chris_christie 1868 42 0.022484
mike_pence 1302 246 0.188940
donald_trump 27493 7357 0.267595
marco_rubio 4407 797 0.180849
gary_johnson 354 56 0.158192
john_kasich 4023 319 0.079294
rick_santorum 245 17 0.069388
ted_cruz 5873 1041 0.177252
barack_obama 3888 920 0.236626


In [388]:
total_u

80849

In [391]:
float(total_c) / 310

62.170967741935485

'barack_obama/barack_obama_2'

In [366]:
np.array(values)[order]

array([ 1,  3,  3,  4,  5,  5,  6,  7,  8, 10, 17, 20, 23, 40, 72, 86])

In [360]:
[(k,counts[k]) for k in keys]

[('jeb_bush', 5),
 ('joe_biden', 7),
 ('hilary_clinton', 72),
 ('bernie_sanders', 40),
 ('rand_paul', 4),
 ('carly_fiorina', 5),
 ('bill_clinton', 8),
 ('chris_christie', 3),
 ('mike_pence', 6),
 ('donald_trump', 86),
 ('marco_rubio', 20),
 ('gary_johnson', 3),
 ('john_kasich', 17),
 ('rick_santorum', 1),
 ('ted_cruz', 23),
 ('barack_obama', 10)]

In [217]:
phrase_audio_feats = s.get_phrase_audio_features()

In [220]:
phrase_audio_feats[10]

{'internal_silence': -1.0693530204840014,
 'max_energy': -0.27726322,
 'max_pitch': -0.16259487,
 'mean_energy': -0.027933361,
 'mean_pitch': 0.39010763,
 'min_energy': 0.21034582,
 'min_pitch': 0.0,
 'range_energy': -0.29338768,
 'range_pitch': -0.16259487,
 'std_energy': -0.81988597,
 'std_pitch': -0.82883531}

In [219]:
cats = {}
for k in phrase_audio_feats[0].keys():
    cats[k] = []
for f in phrase_audio_feats:
    for c in cats:
        cats[c].append(f[c])
means = {}
devs = {}
for k in cats.keys():
    means[k] = np.mean(cats[k])
    devs[k] = np.std(cats[k])
    
for f in phrase_audio_feats:
    for k in cats.keys():
        f[k] -= means[k]
        if devs[k] != 0:
            f[k] /= devs[k]

In [233]:
reload(speech)
s= speech.Speech('hilary_clinton_94')

In [236]:
s.phrase_audio_features

[{'internal_silence': 1.0252379059947219,
  'max_energy': -1.4680401,
  'max_pitch': -2.1974638,
  'mean_energy': 1.0324765,
  'mean_pitch': -1.5669148,
  'min_energy': 1.9720827,
  'min_pitch': 0.0,
  'range_energy': -1.6798016,
  'range_pitch': -2.1974638,
  'std_energy': -1.4055945,
  'std_pitch': -1.7920786},
 {'internal_silence': 0.88352131117379873,
  'max_energy': -0.28310025,
  'max_pitch': 1.0879167,
  'mean_energy': 1.8735211,
  'mean_pitch': -0.52128232,
  'min_energy': 3.4222355,
  'min_pitch': 0.0,
  'range_energy': -0.77184236,
  'range_pitch': 1.0879167,
  'std_energy': -1.3194878,
  'std_pitch': 0.94687879},
 {'internal_silence': -0.063377201046082146,
  'max_energy': 0.30285221,
  'max_pitch': 0.72701865,
  'mean_energy': 0.66706252,
  'mean_pitch': 0.2286,
  'min_energy': -0.66714555,
  'min_pitch': 0.0,
  'range_energy': 0.38486695,
  'range_pitch': 0.72701865,
  'std_energy': 0.24091077,
  'std_pitch': 0.97134107},
 {'internal_silence': -1.4245705750674009,
  'max_e

In [234]:
s.phrase_audio_features

[{'internal_silence': 0.56164383561643838,
  'max_energy': 0.16206369,
  'max_pitch': 52.751198,
  'mean_energy': 0.10038508,
  'mean_pitch': 19.53491,
  'min_energy': 0.021743808,
  'min_pitch': -1.0,
  'range_energy': 0.14031988,
  'range_pitch': 53.751198,
  'std_energy': 0.038469348,
  'std_pitch': 23.380373},
 {'internal_silence': 0.53944020356234101,
  'max_energy': 0.22135666,
  'max_pitch': 190.08621,
  'mean_energy': 0.11274809,
  'mean_pitch': 33.715446,
  'min_energy': 0.033032931,
  'min_pitch': -1.0,
  'range_energy': 0.18832374,
  'range_pitch': 191.08621,
  'std_energy': 0.039254308,
  'std_pitch': 44.591888},
 {'internal_silence': 0.39108364599426176,
  'max_energy': 0.25067702,
  'max_pitch': 175.0,
  'mean_energy': 0.095013641,
  'mean_pitch': 43.885113,
  'min_energy': 0.0011979881,
  'min_pitch': -1.0,
  'range_energy': 0.24947903,
  'range_pitch': 176.0,
  'std_energy': 0.053479105,
  'std_pitch': 44.781334},
 {'internal_silence': 0.17781690140845072,
  'max_energy

In [235]:
s.normalize_phrase_audio_features()

In [214]:
devs['min_pitch'] ==0

True

In [171]:
with open('/data/jrgillick/x_train_deltas.pkl','wb') as f:
    pickle.dump(X_train,f)
    
with open('/data/jrgillick/x_test_deltas.pkl','wb') as f:
    pickle.dump(X_test,f)

with open('/data/jrgillick/y_train_deltas.pkl','wb') as f:
    pickle.dump(y_train,f)
    
with open('/data/jrgillick/y_test_deltas.pkl','wb') as f:
    pickle.dump(y_test,f)

In [217]:
#model = models.train_cv_logistic_regression(X_train,y_train,penalty='l1')

In [15]:
with open('/data/jrgillick/log_reg_model.pkl','rb') as f:
    model = pickle.load(f)

In [42]:
# 3 phrases with deltas
models.evaluate_model(model, X_test, y_test)

Accuracy: 0.657 +/- 0.011 (5014/7633) | Precision: 0.672 | Recall: 0.637 | F1: 0.654


(0.6568845801126687,
 0.67209997283346912,
 0.63664436438497174,
 0.65389189903528477)

In [167]:
# 5 phrases with deltas
models.evaluate_model(model, X_test, y_test)

Accuracy: 0.665 +/- 0.011 (5009/7534) | Precision: 0.665 | Recall: 0.652 | F1: 0.658


(0.6648526679054951,
 0.66529942575881873,
 0.65158007498660953,
 0.65836828575294271)

In [177]:
# 5 phrases with deltas
models.evaluate_model(model, X_test, y_test)

Accuracy: 0.663 +/- 0.011 (4998/7534) | Precision: 0.669 | Recall: 0.635 | F1: 0.652


(0.6633926201221131,
 0.66901805869074493,
 0.63497589716122116,
 0.65155262434734806)

In [162]:
feature_names = np.array(list(featurizer.Featurizer(s).get_combined_feature_names()))# + ['vec_' + str(i) for i in range(4800)])
print len(feature_names)
all_feature_names = []
for i in range(3):
    for prefix in ['t','delta']:
        for f in feature_names:
            all_feature_names.append(prefix+'_'+str(i)+'_'+f)
len(all_feature_names)

155


930

In [164]:
with open('all_feature_names.pkl','wb') as f:
    pickle.dump(all_feature_names,f)

In [178]:
coefs = model.best_estimator_.coef_[0]

In [169]:
len(coefs)

940

In [179]:
zip(np.array(all_feature_names)[np.argsort(coefs)],coefs[np.argsort(coefs)])

[('t_0_std_pitch', -0.19083710199216203),
 ('t_0_min_phone_length', -0.17395552764903666),
 ('t_0_mean_pitch', -0.11530156278929017),
 ('t_0_LIWC_focuspast', -0.1129578905846733),
 ('t_0_range_energy', -0.081952255753081224),
 ('t_1_max_energy', -0.07173065667961652),
 ('t_0_LIWC_function', -0.06703763380566477),
 ('t_0_LIWC_tentat', -0.063148619071874551),
 ('t_1_LIWC_focuspast', -0.058994597390279221),
 ('t_0_LIWC_shehe', -0.052575618144318888),
 ('delta_0_LIWC_body', -0.05044552552647668),
 ('delta_0_min_energy', -0.049535263686345921),
 ('t_2_min_phone_length', -0.049503114718906858),
 ('t_0_LIWC_they', -0.040985158127851001),
 ('t_1_min_phone_length', -0.040576744281923566),
 ('t_4_mean_energy', -0.039017391089718717),
 ('t_3_mean_energy', -0.038743063629010116),
 ('t_1_std_pitch', -0.034930645003831348),
 ('t_0_LIWC_adverb', -0.034442550014040785),
 ('t_0_max_phone_length', -0.034181674107013713),
 ('t_2_LIWC_cause', -0.03171745920187747),
 ('t_4_LIWC_insight', -0.029943111713562

In [182]:
s = speeches[191]
s.file_path

'marco_rubio/marco_rubio_16'

In [184]:
s.alignment.get_phrase_text()[1]

u'GUYS HEAR'

In [33]:
phrases = s.alignment.get_phrase_text()
s_labels = featurizer.Featurizer(s).get_binary_labels()
times = s.alignment.get_phrase_times()
feature_list = featurizer.Featurizer(s).get_all_features()

In [34]:
s_feats, s_labs = models.format_multiple_phrase_input_with_deltas([feature_list],[s_labels],phrase_count=3)

100%|██████████| 410/410 [00:00<00:00, 13928.02it/s]


In [205]:
i = 83
print("Prediction: " + str(model.predict_proba(scaler.transform(s_feats[i-2]))[0][1]))
print("True label: %d" % s_labs[i-2])
phrases[i-2:i+1]

Prediction: 0.516284279539
True label: 0


/home/jrgillick/.local/lib/python2.7/site-packages/sklearn/preprocessing/data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/home/jrgillick/.local/lib/python2.7/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


[u'I SAID',
 u"SAID BECAUSE YOU HAVE YOUR TURN THERE'S A LINE I SAID I DIDN'T KNOW THERE WAS A LINE BUT WHAT",
 u'ARE WE WAITING FOR AFTER SEVEN YEARS OF OBAMA THIS IS NOT A TIME FOR PATIENCE THIS IS A TIME FOR URGENT ACTION BECAUSE IF WE GET THIS ELECTION WRONG']

In [139]:
Audio(s.voice_y[int(times[i][0]*s.sr):int((times[i][1]+2)*s.sr)],rate=s.sr)

In [140]:
Audio(s.crowd_y[int(times[i][0]*s.sr):int((times[i][1]+2)*s.sr)],rate=s.sr)

In [155]:
zip(s_feats[i-2],feature_names)

[(0.56992393212404913, 'internal_silence'),
 (0.18815326690673828, 'max_energy'),
 (186.86441040039062, 'max_pitch'),
 (0.054628688842058182, 'mean_energy'),
 (24.704828262329102, 'mean_pitch'),
 (0.0020158104598522186, 'min_energy'),
 (-1.0, 'min_pitch'),
 (0.18613745272159576, 'range_energy'),
 (187.86441040039062, 'range_pitch'),
 (0.041984330862760544, 'std_energy'),
 (33.330783843994141, 'std_pitch'),
 (0.20000000000000001, 'alliteration'),
 (0.47999999999999998, 'homogeneity'),
 (0.20000000000000001, 'plosive'),
 (0.22, 'rhyme'),
 (0.0, 'LIWC_achiev'),
 (1.0, 'LIWC_adj'),
 (0.0, 'LIWC_adverb'),
 (1.0, 'LIWC_affect'),
 (0.0, 'LIWC_affiliation'),
 (0.0, 'LIWC_anger'),
 (0.0, 'LIWC_anx'),
 (1.0, 'LIWC_article'),
 (0.0, 'LIWC_assent'),
 (1.0, 'LIWC_auxverb'),
 (1.0, 'LIWC_bio'),
 (0.0, 'LIWC_body'),
 (0.0, 'LIWC_cause'),
 (0.0, 'LIWC_certain'),
 (1.0, 'LIWC_cogproc'),
 (1.0, 'LIWC_compare'),
 (0.0, 'LIWC_conj'),
 (0.0, 'LIWC_death'),
 (0.0, 'LIWC_differ'),
 (0.0, 'LIWC_discrep'),
 (1

In [146]:
times[i][1]

323.74999899999995

In [144]:
s.applause_list.get_instances()

[(9.10125, 17.11128),
 (25.3767, 32.24907),
 (62.45501, 66.19302),
 (75.75862, 79.9842),
 (89.50337, 96.28287),
 (100.27628, 104.57151),
 (109.37753, 112.81371),
 (140.25677, 163.65999),
 (167.02652, 169.53401),
 (177.72978, 182.83762),
 (185.41476, 189.19921),
 (199.99534, 206.63553),
 (214.25087, 216.73514),
 (218.17462, 225.16308),
 (229.80657, 239.51148),
 (267.95289, 271.78377),
 (274.84848, 279.51519),
 (283.71755, 286.15539),
 (291.37932, 293.46889),
 (295.72099, 302.17545),
 (307.44581, 318.70629),
 (319.1242, 322.93187),
 (323.95344, 328.31832),
 (344.15264, 351.9305),
 (355.62208, 359.80122),
 (360.89244, 367.37012),
 (462.09742, 468.73762),
 (509.94864, 516.91388),
 (524.92391, 530.72828),
 (532.12133, 534.41986),
 (563.78996, 572.82156),
 (591.34911, 601.35584),
 (611.91979, 613.93971),
 (663.78763, 666.01651),
 (674.23549, 693.18095),
 (723.99054, 726.4516),
 (730.86292, 735.48319),
 (737.68885, 739.82486),
 (760.44198, 769.75219),
 (785.16859, 793.34114),
 (802.25665, 804

In [ ]:
featurizer.Featurizer(s).get_binary_labels()

In [46]:
len(feature_list)

412

In [109]:
s.load_librosa()
s.load_librosa_crowd()
s.load_librosa_voice()

True

In [114]:
s.applause_list.get_instances()

[(22.77613, 33.10781),
 (35.40632, 40.42124),
 (54.28195, 59.48261),
 (77.45277, 81.09788),
 (90.89556, 95.30684),
 (139.51249, 144.2256),
 (152.44451, 155.64849),
 (166.35164, 170.32179),
 (184.97188, 188.66343),
 (206.84254, 213.32015),
 (295.9968, 298.38818),
 (418.09635, 420.51095),
 (513.5425, 517.69839),
 (642.4447, 650.59396),
 (689.18104, 694.70675),
 (716.06662, 718.20261),
 (743.11473, 747.40992),
 (804.75654, 811.4199),
 (883.32374, 893.4929),
 (922.32873, 925.39341),
 (930.01364, 933.0551),
 (935.609, 942.80635),
 (1002.75331, 1008.27901),
 (1042.40838, 1045.2641),
 (1050.69694, 1058.61403),
 (1060.44819, 1068.31884),
 (1107.02201, 1111.24755),
 (1147.69864, 1152.92252),
 (1168.19948, 1173.6091),
 (1212.03366, 1214.68042),
 (1290.74016, 1294.87283),
 (1308.8264, 1311.47317),
 (1319.29738, 1322.45493),
 (1334.59755, 1339.03205),
 (1394.49808, 1401.95082),
 (1476.68716, 1479.47323),
 (1486.48485, 1495.56279),
 (1537.93429, 1547.96414),
 (1777.90785, 1788.611),
 (1963.5066, 19

In [72]:
np.max(s.get_rmse_at_times(times[i][0],times[i][1]))

0.11237981

In [82]:
def get_relative_rmse(s, start_time, end_time):
    mean_rmse = np.mean(s.rmse)
    return (np.mean(s.get_rmse_at_times(start_time, end_time)) / mean_rmse)

In [143]:
zip(np.array(all_feature_names)[np.argsort(coefs)],coefs[np.argsort(coefs)])

[('delta_1_min_energy', -5.3331412176435844),
 ('t_range_energy', -3.7987877529767142),
 ('delta_1_mean_energy', -3.2876116221160498),
 ('delta_1_max_energy', -2.6744090819347108),
 ('delta_2_mean_energy', -2.591703157930854),
 ('delta_2_min_energy', -1.5493668703809391),
 ('t-1_range_energy', -1.1400554998324293),
 ('t-2_max_energy', -1.067698892577204),
 ('t-2_min_energy', -0.84401951551697285),
 ('delta_2_max_energy', -0.63299062651404325),
 ('delta_1_applause_feature', -0.45032232456843896),
 ('t-1_max_energy', -0.43470826606265861),
 ('delta_1_homogeneity', -0.40243126603563639),
 ('t-2_mean_energy', -0.34277295451187861),
 ('t_LIWC_shehe', -0.24083157646188816),
 ('t_LIWC_function', -0.23940568192229625),
 ('t_LIWC_death', -0.22860996088981517),
 ('t-2_range_energy', -0.22368004587993517),
 ('t_LIWC_focuspast', -0.21149386259085526),
 ('t-1_LIWC_shehe', -0.20404594155490416),
 ('delta_1_LIWC_filler', -0.19736223456553884),
 ('t-2_LIWC_shehe', -0.19305332987880999),
 ('t_LIWC_see'

In [148]:
x=[('t_thank_you_feature', 0.3774252899630714),
('t-1_homogeneity', 0.38359345291463331),
('t_internal_silence', 0.51532797033195787),
('t-1_min_energy', 0.7053473548642436),
('t_homogeneity', 0.78602471894641057),
 ('t-1_applause_feature', 0.83723373483706343),
 ('delta_2_std_energy', 0.87771915384597043),
 ('delta_2_range_energy', 0.91637545394360398),
 ('t-2_applause_feature', 1.0031853932975721),
 ('t_applause_feature', 1.2875560594055036),
 ('t_max_energy', 2.2397008158732721),
 ('t-1_mean_energy', 2.2489302034200751),
 ('delta_1_std_energy', 2.3550216311442367),
 ('delta_1_range_energy', 2.6587322531427282),
 ('t-1_std_energy', 2.7222193509839596),
 ('t-2_std_energy', 3.599938504830182),
 ('t_mean_energy', 5.5365418255368963),
 ('t_min_energy', 6.0384885725075756)]

In [151]:
x

[('t_min_energy', 6.038488572507576),
 ('t_mean_energy', 5.536541825536896),
 ('t-2_std_energy', 3.599938504830182),
 ('t-1_std_energy', 2.7222193509839596),
 ('delta_1_range_energy', 2.658732253142728),
 ('delta_1_std_energy', 2.3550216311442367),
 ('t-1_mean_energy', 2.248930203420075),
 ('t_max_energy', 2.239700815873272),
 ('t_applause_feature', 1.2875560594055036),
 ('t-2_applause_feature', 1.003185393297572),
 ('delta_2_range_energy', 0.916375453943604),
 ('delta_2_std_energy', 0.8777191538459704),
 ('t-1_applause_feature', 0.8372337348370634),
 ('t_homogeneity', 0.7860247189464106),
 ('t-1_min_energy', 0.7053473548642436),
 ('t_internal_silence', 0.5153279703319579),
 ('t-1_homogeneity', 0.3835934529146333),
 ('t_thank_you_feature', 0.3774252899630714)]

In [137]:
np.argsort(coefs)

array([275,   7, 273, 271, 363, 365,  97, 181, 185, 361, 358,  91, 282,
       183,  77,  44,  32, 187,  41, 167, 309, 257,  75, 270, 134, 122,
        46,  84, 131, 359, 136,  82, 165,  30,  37, 399, 172, 127, 221,
       373, 224, 120, 296,  81, 360,  51, 262,  50,  60,  17, 193, 310,
       247,  47, 421,  80, 141, 449, 236, 125, 264, 158, 210, 174, 171,
        69, 423, 342, 319,  29, 225, 206, 355, 371, 308, 146, 230, 140,
        35, 107, 215, 372, 135, 427, 231, 294, 324, 430, 213, 298, 376,
        62, 139, 159, 255, 197,  56, 266, 313, 198, 176, 124, 386, 253,
        68, 436, 431, 425, 383, 119, 400, 346, 217, 398, 315, 414, 233,
       207,  86, 445, 248, 335, 433, 413, 108, 415, 212, 338, 202, 123,
       289,  53,  78, 387, 133, 379, 226, 137,  10, 157,  34, 214, 384,
       426, 261, 209, 393, 353, 250, 284, 116, 229, 150, 288,  31,  25,
        45,  33, 112, 375, 331, 104, 100, 447, 249, 304, 337, 432, 328,
       326, 388, 343, 306, 190, 305,  22, 285, 349,  52, 378, 41

In [127]:
# 3 phrases with no deltas
models.evaluate_model(model, X_test, y_test)

Accuracy: 0.647 +/- 0.011 (4941/7633) | Precision: 0.650 | Recall: 0.631 | F1: 0.640


(0.6473208437049652,
 0.65021691973969631,
 0.63105263157894742,
 0.64049145299145305)

In [84]:
s = speeches[81]

In [85]:
s.file_path

'donald_trump/donald_trump_33'

In [86]:
text = s.alignment.get_phrase_text()
times = s.alignment.get_phrase_times()

In [87]:
end_labels = [s.alignment.applause_follows(t[1]) for t in times]

In [88]:
zip(text,end_labels)

[(u'YOU EVERYBODY THANK YOU', True),
 (u'ARE GOING TO MAKE WE ARE', True),
 (u'AM THRILLED TO BE HERE TONIGHT BEAUTIFUL CITY MANNHEIM', True),
 (u'WILL TELL YOU WHAT WE WIN THE GREAT STATE OF PENNSYLVANIA I WENT TO SCHOOL IN PENNSYLVANIA',
  True),
 (u'ARE GOING TO WIN PENNSYLVANIA WE ARE GOING TO WIN BACK THE WHITE HOUSE AND WE ARE GOING TO BE SO HAPPY WE ARE GOING TO BE SO HAPPY AND WE GOING TO AGAIN BE PROUD OF OUR COUNTRY WE WILL BE PROUD WE GOING TO TAKE ON THE CORRUPT MEDIA THE POWERFUL LOBBYISTS AND THE SPECIAL INTERESTS THAT HAVE STOLEN YOUR JOBS HERE FACTORIES IN YOUR FUTURE THAT',
  False),
 (u'WHAT HAS HAPPENED WE ARE GOING TO STOP HILLARY CLINTON FROM CONTINUING TO RATE THE INDUSTRY FROM YOUR STATE FOR HER PROFIT',
  True),
 (u'CLINTON HAS COLLECTED MILLIONS OF DOLLARS FROM THE SAME GLOBAL CORPORATIONS SHIPPING YOUR JOBS YOUR DREAMS TO OTHER COUNTRIES YOU KNOW IT AND EVERYBODY ELSE KNOWS IT',
  False),
 (u'IS WHY CLINTON', False),
 (u'100 APPROVE TRANSPACIFIC', False),
 (u'

In [83]:
np.sum(end_labels)

7

In [147]:
x

NameError: name 'x' is not defined